In [1]:
import pandas as pd
import xarray as xr
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.smpickle import load_pickle

import comp_utils
from comp_utils import day_ahead_market_times
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os
from datetime import datetime, timedelta
import pickle as pkl

In [2]:
with open('proxies.txt', 'r') as file:
        proxy = file.read()
proxies = {'http': proxy, 'https': proxy}

In [3]:
rebase_api_client = comp_utils.RebaseAPI(api_key = open("A-Team_key.txt").read(), proxy=proxies)

In [69]:
def update_day_ahead_price_csv(min_date, csv_path='day_ahead_price.csv'):
    """
    Fetches day-ahead prices from the Rebase API and appends them to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['settlement_date'].max()
                # df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'price'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'price'])
            print("Creating new CSV file")
        
        # Generate dates to updat
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                prices = rebase_api_client.get_day_ahead_price(date_str)
                if len(prices):
                    print(f"Fetched {len(prices)} prices for {date_str}")
                    new_df = pd.DataFrame(prices)
                    # new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [80]:
def update_market_index_csv(min_date, csv_path='market_index.csv'):
    """
    Fetches market index data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'index'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'index'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                indices = rebase_api_client.get_market_index(date_str)
                if len(indices):
                    print(f"Fetched {len(indices)} indices for {date_str}")
                    new_df = pd.DataFrame(indices)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [82]:
def update_imbalance_price_csv(min_date, csv_path='imbalance_price.csv'):
    """
    Fetches imbalance prices from the Rebase API and appends them to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'price'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'price'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                prices = rebase_api_client.get_imbalance_price(date_str)
                if len(prices):
                    print(f"Fetched {len(prices)} prices for {date_str}")
                    new_df = pd.DataFrame(prices)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [84]:
def update_wind_total_prduction_csv(min_date, csv_path='wind_total_production.csv'):
    """
    Fetches total wind production data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'production'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'production'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                productions = rebase_api_client.get_wind_total_production(date_str)
                if len(productions):
                    print(f"Fetched {len(productions)} productions for {date_str}")
                    new_df = pd.DataFrame(productions)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [86]:
def update_solar_total_prodcution_csv(min_date, csv_path='solar_total_production.csv'):
    """
    Fetches total solar production data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'production'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'production'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                productions = rebase_api_client.get_solar_total_production(date_str)
                if len(productions):
                    print(f"Fetched {len(productions)} productions for {date_str}")
                    new_df = pd.DataFrame(productions)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [94]:
def update_solar_wind_forecast_csv(min_date, csv_path='solar_wind_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        min_date = datetime.strptime(min_date, '%Y-%m-%d').date()
        today = datetime.today().date()
        
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                min_date = df['timestamp_utc'].max()
                df['timestamp_utc'] = pd.to_datetime(df['timestamp_utc'])
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'solar_forecast', 'wind_forecast'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'solar_forecast', 'wind_forecast'])
            print("Creating new CSV file")
        
        # Generate dates to update
        print(f"min_date: {min_date}, today: {today}")
        dates_to_update = pd.date_range(start=min_date, end=today, freq='D')
        try:
            for date in dates_to_update:
                date_str = date.strftime('%Y-%m-%d')
                forecasts = rebase_api_client.get_solar_wind_forecast(date_str)
                if len(forecasts):
                    print(f"Fetched {len(forecasts)} forecasts for {date_str}")
                    new_df = pd.DataFrame(forecasts)
                    new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                    df = pd.concat([df, new_df], ignore_index=True)
                else:
                    print(f"No data returned for {date_str}")
            df.drop_duplicates(subset=['timestamp_utc'], inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for {date_str}: {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [9]:
update_solar_wind_forecast_csv('2024-05-01', 'solar_wind_forecast.csv')

NameError: name 'update_solar_wind_forecast_csv' is not defined

In [18]:
def update_margin_forecast(csv_path='margin_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['forecast_date', 'publish_time_utc', 'margin'])
        else:
            df = pd.DataFrame(columns=['forecast_date', 'publish_time_utc', 'margin'])
            print("Creating new CSV file")
        
        # Generate dates to update
        try:
            forecasts = rebase_api_client.get_margin_forecast()
            if len(forecasts):
                print(f"Fetched {len(forecasts)} forecasts")
                new_df = pd.DataFrame(forecasts)
                # new_df['timestamp_utc'] = pd.to_datetime(new_df['timestamp_utc'])
                df = pd.concat([df, new_df], ignore_index=True)
            else:
                print(f"No data returned for")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for : {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [4]:
def update_day_ahead_demand_forecast(csv_path='day_ahead_demand_forecast.csv'):
    """
    Fetches solar and wind forecast data from the Rebase API and appends it to a CSV file.
    The CSV file is created if it does not exist.
    :param min_date: The earliest date to fetch data for, in the format 'YYYY-MM-DD'
    :param csv_path: The path to the CSV file to read/write
    :return: None
    """
    try:
        # Initialize an empty DataFrame
        if os.path.exists(csv_path):
            try:
                df = pd.read_csv(csv_path)
                print(f"Loaded existing CSV with {len(df)} rows")
            except Exception as e:
                print(f"Error reading existing CSV: {e}")
                df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
                    'publish_time_utc', 'transmission_system_demand', 'national_demand'])
        else:
            df = pd.DataFrame(columns=['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
                'publish_time_utc', 'transmission_system_demand', 'national_demand'])
            print("Creating new CSV file")
        
        # Generate dates to update
        try:
            forecasts = rebase_api_client.get_day_ahead_demand_forecast()
            if len(forecasts):
                print(f"Fetched {len(forecasts)} forecasts")
                new_df = pd.DataFrame(forecasts)
                df = pd.concat([df, new_df], ignore_index=True)
            else:
                print(f"No data returned for")
            df.drop_duplicates(inplace=True)
            df.to_csv(csv_path, index=False)
        except Exception as e:
            print(f"Error fetching data for : {e}")
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [19]:
update_margin_forecast()

Loaded existing CSV with 13 rows


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>
Fetched 13 forecasts


In [5]:
update_day_ahead_demand_forecast()

Creating new CSV file


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>
Fetched 79 forecasts


In [112]:
rebase_api_client.get_solar_wind_forecast('2024-09-01').columns

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Index(['timestamp_utc', 'settlement_date', 'settlement_period', 'solar_mw',
       'wind_offshore_mw', 'wind_onshore_mw'],
      dtype='object')

In [20]:
df1 = pd.DataFrame(rebase_api_client.get_day_ahead_demand_forecast())

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>


In [22]:
df1

,timestamp_utc,settlement_date,settlement_period,boundary,publish_time_utc,transmission_system_demand,national_demand
0,2024-10-02T12:30:00Z,2024-10-02,28,N,2024-10-02T12:15:00Z,29658.0,28132.0
1,2024-10-02T13:00:00Z,2024-10-02,29,N,2024-10-02T12:15:00Z,29689.0,28191.0
2,2024-10-02T13:30:00Z,2024-10-02,30,N,2024-10-02T12:15:00Z,29547.0,28049.0
3,2024-10-02T14:00:00Z,2024-10-02,31,N,2024-10-02T12:15:00Z,29443.0,27990.0
4,2024-10-02T14:30:00Z,2024-10-02,32,N,2024-10-02T12:15:00Z,30304.0,28824.0
...,...,...,...,...,...,...,...
74,2024-10-04T01:30:00Z,2024-10-04,6,N,2024-10-02T12:15:00Z,21471.0,20689.0
75,2024-10-04T02:00:00Z,2024-10-04,7,N,2024-10-02T12:15:00Z,21192.0,20410.0
76,2024-10-04T02:30:00Z,2024-10-04,8,N,2024-10-02T12:15:00Z,21043.0,20261.0
77,2024-10-04T03:00:00Z,2024-10-04,9,N,2024-10-02T12:15:00Z,20901.0,20120.0


In [21]:
df1.columns

Index(['timestamp_utc', 'settlement_date', 'settlement_period', 'boundary',
       'publish_time_utc', 'transmission_system_demand', 'national_demand'],
      dtype='object')

In [10]:
times = day_ahead_market_times()

In [14]:
times

DatetimeIndex(['2024-10-02 23:00:00+01:00', '2024-10-02 23:30:00+01:00',
               '2024-10-03 00:00:00+01:00', '2024-10-03 00:30:00+01:00',
               '2024-10-03 01:00:00+01:00', '2024-10-03 01:30:00+01:00',
               '2024-10-03 02:00:00+01:00', '2024-10-03 02:30:00+01:00',
               '2024-10-03 03:00:00+01:00', '2024-10-03 03:30:00+01:00',
               '2024-10-03 04:00:00+01:00', '2024-10-03 04:30:00+01:00',
               '2024-10-03 05:00:00+01:00', '2024-10-03 05:30:00+01:00',
               '2024-10-03 06:00:00+01:00', '2024-10-03 06:30:00+01:00',
               '2024-10-03 07:00:00+01:00', '2024-10-03 07:30:00+01:00',
               '2024-10-03 08:00:00+01:00', '2024-10-03 08:30:00+01:00',
               '2024-10-03 09:00:00+01:00', '2024-10-03 09:30:00+01:00',
               '2024-10-03 10:00:00+01:00', '2024-10-03 10:30:00+01:00',
               '2024-10-03 11:00:00+01:00', '2024-10-03 11:30:00+01:00',
               '2024-10-03 12:00:00+01:00', '2024-1

In [26]:
formatted_dates = [dt.strftime('%Y-%m-%d') for dt in times]
formatted_dates

['2024-10-02',
 '2024-10-02',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03',
 '2024-10-03']

In [27]:
for i in formatted_dates:
    rebase_api_client.get_day_ahead_price(i)

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[]


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[]


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

[]


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[]


c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

In [29]:
rebase_api_client.get_day_ahead_price(day="2023-09-29").columns

c:\Temp\EnergyProject\HEFTcom24\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'srv01prox.corp.trumpf.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


[{'timestamp_utc': '2023-09-28T22:00:00Z', 'settlement_date': '2023-09-28', 'settlement_period': 47, 'price': 65.96}, {'timestamp_utc': '2023-09-28T22:30:00Z', 'settlement_date': '2023-09-28', 'settlement_period': 48, 'price': 65.96}, {'timestamp_utc': '2023-09-28T23:00:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 1, 'price': 47.99}, {'timestamp_utc': '2023-09-28T23:30:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 2, 'price': 47.99}, {'timestamp_utc': '2023-09-29T00:00:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 3, 'price': 26.51}, {'timestamp_utc': '2023-09-29T00:30:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 4, 'price': 26.51}, {'timestamp_utc': '2023-09-29T01:00:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 5, 'price': 25.95}, {'timestamp_utc': '2023-09-29T01:30:00Z', 'settlement_date': '2023-09-29', 'settlement_period': 6, 'price': 25.95}, {'timestamp_utc': '2023-09-29T02:00:00Z', 'settlement_date': '2023-09

Index(['timestamp_utc', 'settlement_date', 'settlement_period', 'price'], dtype='object')

In [ ]:
rebase_api_client.get_hornsea_dwd()

In [6]:
rebase_api_client.get_day_ahead_price(day="2024-09-30")

ConnectTimeout: HTTPSConnectionPool(host='api.rebase.energy', port=443): Max retries exceeded with url: /challenges/data/day_ahead_price?day=2024-09-30 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001961D7E5010>, 'Connection to api.rebase.energy timed out. (connect timeout=None)'))

In [ ]:
rebase_api_client.get_day_ahead_price(day="2024-09-30")

In [ ]:
latest_dwd_Hornsea1 = comp_utils.weather_df_to_xr(rebase_api_client.get_hornsea_dwd())
latest_dwd_solar = comp_utils.weather_df_to_xr(rebase_api_client.get_pes10_nwp("DWD_ICON-EU"))

In [ ]:
test_date = "2023-05-05"

rebase_api_client.get_variable(day=test_date,variable="day_ahead_price").head()

In [5]:
rebase_api_client.get_variable(day=test_date,variable="imbalance_price").head()
# rebase_api_client.get_variable(day=test_date,variable="wind_total_production").head()
# rebase_api_client.get_variable(day=test_date,variable="solar_total_production").head()

ValueError: DataFrame constructor not properly called!

In [4]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/weather/v2/query"
headers = {"Authorization": API_key}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-30',
    'end-date': '2024-10-03',
    'reference-time-freq': '6H',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed',
    'output-format': 'json'   
}
response = requests.get(url, headers=headers, params=params)


In [5]:
response.json()

{'message': 'Unauthorized'}

In [29]:
import requests

your_api_key = '' # insert your API key here
url = "https://api.rebase.energy/weather/v2/query"
headers = {"Authorization": API_key,
           'Content-Type': 'application/json'}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-30',
    'end-date': '2024-10-03',
    'reference-time-freq': '6H',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed',
    'output-format': 'json'
}
response = requests.post(url,
                         headers=headers,
                         json=params,)
                        #  content_type='application/json')

In [31]:
response.json()

{'message': 'Unauthorized'}

In [32]:
if response.status_code == 200:
    print("Request successful.")
    print(response.json())  # Print the JSON response
else:
    print(f"Request failed with status code {response.status_code}: {response.text}")

Request failed with status code 401: {"message":"Unauthorized"}



In [33]:
response.json()

{'message': 'Unauthorized'}

In [39]:
import requests

your_api_key = '' # insert your API key here
url = "https://api.rebase.energy/weather/v2/point/historical"
headers = {"Authorization": your_api_key}
params = {
    'model': 'DWD_ICON-EU',
    'start-date': '2024-09-01',
    'end-date': '2024-09-09',
    'reference-time-freq': '6H',
    'forecast-horizon': 'full',
    'latitude': '60.1, 61.2, 59.33', 
    'longitude': '17.2, 13.1, 18.05',
    'variables': 'Temperature, WindSpeed:10'
}
response = requests.get(url, headers=headers, params=params)

In [40]:
response.json()

{'message': 'Unauthorized'}

In [51]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/day_ahead_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [52]:
response.json()

'Bearer token is invalid'

In [53]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/market_index"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [54]:
response.json()

'Bearer token is invalid'

In [55]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/imbalance_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)

In [56]:
response.json()

'Bearer token is invalid'

In [ ]:
import requests
API_key = open("team_key.txt").read()
url = "https://api.rebase.energy/challenges/data/imbalance_price"
headers = {"Authorization": f"Bearer {API_key}"}  # Prefix the API key with 'Bearer '
params = {
    "date": "2024-10-01"
}
response = requests.get(url, headers=headers, params=params)